## Imports and CSV

In [44]:
import nltk
nltk.download("punkt_tab")
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer, TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.stem import PorterStemmer
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from textblob import TextBlob
import vaderSentiment as vs
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer()
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score
from sklearn.metrics import classification_report
import pickle

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\evios\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\evios\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
wnl = WordNetLemmatizer()

In [4]:
wine = pd.read_csv(r'D:\GIT\classification_project\NotesM2\Files\winemag-data_first150k.csv', encoding = 'latin1', sep = ',')

In [5]:
desc = wine['description']

In [6]:
wine

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum SelecciÃ³n Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen RodrÃ­guez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La BÃ©gude, named af...",La BrÃ»lade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la BÃ©gude
...,...,...,...,...,...,...,...,...,...,...,...
150925,150925,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio
150926,150926,France,"Offers an intriguing nose with ginger, lime an...",CuvÃ©e Prestige,91,27.0,Champagne,Champagne,NaN,Champagne Blend,H.Germain
150927,150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Terredora
150928,150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut RosÃ©,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset


## Tokenizing/Lemmatizing

In [7]:
wine['tokens'] = wine['description'].apply(lambda x: (WordPunctTokenizer().tokenize(x)))
wine['tokens']

0         [This, tremendous, 100, %, varietal, wine, hai...
1         [Ripe, aromas, of, fig, ,, blackberry, and, ca...
2         [Mac, Watson, honors, the, memory, of, a, wine...
3         [This, spent, 20, months, in, 30, %, new, Fren...
4         [This, is, the, top, wine, from, La, BÃ, ©, gu...
                                ...                        
150925    [Many, people, feel, Fiano, represents, southe...
150926    [Offers, an, intriguing, nose, with, ginger, ,...
150927    [This, classic, example, comes, from, a, cru, ...
150928    [A, perfect, salmon, shade, ,, with, scents, o...
150929    [More, Pinot, Grigios, should, taste, like, th...
Name: tokens, Length: 150930, dtype: object

In [8]:
def lemmatize_list(tokens):
    return [wnl.lemmatize(word) for word in tokens]

In [9]:
wine['cleaned_lemma_description'] = wine['tokens'].apply(lemmatize_list)
wine

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery,tokens,cleaned_lemma_description
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,"[This, tremendous, 100, %, varietal, wine, hai...","[This, tremendous, 100, %, varietal, wine, hai..."
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum SelecciÃ³n Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen RodrÃ­guez,"[Ripe, aromas, of, fig, ,, blackberry, and, ca...","[Ripe, aroma, of, fig, ,, blackberry, and, cas..."
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,"[Mac, Watson, honors, the, memory, of, a, wine...","[Mac, Watson, honor, the, memory, of, a, wine,..."
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,"[This, spent, 20, months, in, 30, %, new, Fren...","[This, spent, 20, month, in, 30, %, new, Frenc..."
4,4,France,"This is the top wine from La BÃ©gude, named af...",La BrÃ»lade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la BÃ©gude,"[This, is, the, top, wine, from, La, BÃ, ©, gu...","[This, is, the, top, wine, from, La, BÃ, ©, gu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150925,150925,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio,"[Many, people, feel, Fiano, represents, southe...","[Many, people, feel, Fiano, represents, southe..."
150926,150926,France,"Offers an intriguing nose with ginger, lime an...",CuvÃ©e Prestige,91,27.0,Champagne,Champagne,NaN,Champagne Blend,H.Germain,"[Offers, an, intriguing, nose, with, ginger, ,...","[Offers, an, intriguing, nose, with, ginger, ,..."
150927,150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Terredora,"[This, classic, example, comes, from, a, cru, ...","[This, classic, example, come, from, a, cru, v..."
150928,150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut RosÃ©,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset,"[A, perfect, salmon, shade, ,, with, scents, o...","[A, perfect, salmon, shade, ,, with, scent, of..."


In [10]:
wine['cleaned_lemma_description'] = wine['cleaned_lemma_description'].apply(lambda x: " ".join(x))
wine

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery,tokens,cleaned_lemma_description
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,"[This, tremendous, 100, %, varietal, wine, hai...",This tremendous 100 % varietal wine hail from ...
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum SelecciÃ³n Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen RodrÃ­guez,"[Ripe, aromas, of, fig, ,, blackberry, and, ca...","Ripe aroma of fig , blackberry and cassis are ..."
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,"[Mac, Watson, honors, the, memory, of, a, wine...",Mac Watson honor the memory of a wine once mad...
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,"[This, spent, 20, months, in, 30, %, new, Fren...","This spent 20 month in 30 % new French oak , a..."
4,4,France,"This is the top wine from La BÃ©gude, named af...",La BrÃ»lade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la BÃ©gude,"[This, is, the, top, wine, from, La, BÃ, ©, gu...","This is the top wine from La BÃ © gude , named..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150925,150925,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio,"[Many, people, feel, Fiano, represents, southe...",Many people feel Fiano represents southern Ita...
150926,150926,France,"Offers an intriguing nose with ginger, lime an...",CuvÃ©e Prestige,91,27.0,Champagne,Champagne,NaN,Champagne Blend,H.Germain,"[Offers, an, intriguing, nose, with, ginger, ,...","Offers an intriguing nose with ginger , lime a..."
150927,150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Terredora,"[This, classic, example, comes, from, a, cru, ...",This classic example come from a cru vineyard ...
150928,150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut RosÃ©,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset,"[A, perfect, salmon, shade, ,, with, scents, o...","A perfect salmon shade , with scent of peach ,..."


## TextBlob

In [11]:
TextBlob(wine.description[0]).sentiment.polarity

0.22407407407407406

In [12]:
wine['textblob'] = wine['description'].apply(lambda description: 1 if TextBlob(description).sentiment[0] > 0 else 0)
wine

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery,tokens,cleaned_lemma_description,textblob
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,"[This, tremendous, 100, %, varietal, wine, hai...",This tremendous 100 % varietal wine hail from ...,1
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum SelecciÃ³n Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen RodrÃ­guez,"[Ripe, aromas, of, fig, ,, blackberry, and, ca...","Ripe aroma of fig , blackberry and cassis are ...",1
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,"[Mac, Watson, honors, the, memory, of, a, wine...",Mac Watson honor the memory of a wine once mad...,1
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,"[This, spent, 20, months, in, 30, %, new, Fren...","This spent 20 month in 30 % new French oak , a...",1
4,4,France,"This is the top wine from La BÃ©gude, named af...",La BrÃ»lade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la BÃ©gude,"[This, is, the, top, wine, from, La, BÃ, ©, gu...","This is the top wine from La BÃ © gude , named...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150925,150925,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio,"[Many, people, feel, Fiano, represents, southe...",Many people feel Fiano represents southern Ita...,1
150926,150926,France,"Offers an intriguing nose with ginger, lime an...",CuvÃ©e Prestige,91,27.0,Champagne,Champagne,NaN,Champagne Blend,H.Germain,"[Offers, an, intriguing, nose, with, ginger, ,...","Offers an intriguing nose with ginger , lime a...",1
150927,150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Terredora,"[This, classic, example, comes, from, a, cru, ...",This classic example come from a cru vineyard ...,1
150928,150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut RosÃ©,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset,"[A, perfect, salmon, shade, ,, with, scents, o...","A perfect salmon shade , with scent of peach ,...",1


In [13]:
wine['textblob'].value_counts()

textblob
1    127294
0     23636
Name: count, dtype: int64

In [14]:
df = pd.DataFrame()
df.insert(0, "Desc", wine.description)
df.insert(1, "TextBlob", wine.textblob)
df


,Desc,TextBlob
0,This tremendous 100% varietal wine hails from ...,1
1,"Ripe aromas of fig, blackberry and cassis are ...",1
2,Mac Watson honors the memory of a wine once ma...,1
3,"This spent 20 months in 30% new French oak, an...",1
4,"This is the top wine from La BÃ©gude, named af...",1
...,...,...
150925,Many people feel Fiano represents southern Ita...,1
150926,"Offers an intriguing nose with ginger, lime an...",1
150927,This classic example comes from a cru vineyard...,1
150928,"A perfect salmon shade, with scents of peaches...",1


In [47]:
pickle.dump(df, open("wine_textblob_graphg.pickle", 'wb'))

## Modelling / CountVectorizer

In [15]:
zeros = wine[wine['textblob'] == 0]
ones = wine[wine['textblob'] == 1]

zero_sample = zeros.sample(n=5000, random_state = 42)
one_sample = ones.sample(n=5000, random_state = 42)

balanced_subset = pd.concat([zero_sample, one_sample])

In [16]:
balanced_subset['textblob'].value_counts()

textblob
0    5000
1    5000
Name: count, dtype: int64

In [17]:
vect = CountVectorizer(stop_words = 'english', lowercase = False)

In [18]:
X = balanced_subset['description'].values
Y = balanced_subset['textblob'].values

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25)

In [49]:
fitted_vect = vect.fit(x_train)

In [50]:
pickle.dump(fitted_vect, open('fitted_vectorizer.pickle', 'wb'))

In [21]:
X_train = vect.transform(x_train)
X_test = vect.transform(x_test)

In [22]:
X_train

<7500x11859 sparse matrix of type '<class 'numpy.int64'>'
	with 177921 stored elements in Compressed Sparse Row format>

In [23]:
x_train_dense = pd.DataFrame(X_train.todense(), columns = sorted(vect.vocabulary_))
x_train_dense

,000,01,02,03,03s,04,05,06,06s,07,...,zesty,zing,zinger,zingy,zip,zippiness,zippy,zone,zu,Â½t
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [25]:
model = lr.fit(x_train_dense, y_train)
model

LogisticRegression()

In [45]:
pickle.dump(model, open('wine_classification_model.pickle', 'wb'))

In [26]:
y_scores = model.predict_proba(X_test)[:, 1]

C:\Users\evios\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [27]:
preds = model.predict(X_test)

In [28]:
pd.Series(preds).value_counts()

0    1326
1    1174
Name: count, dtype: int64

In [29]:
y_test

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

# CHANGE TO CLASSIFICATION SCORES (PRECISION RECALL)

## Model Scores / CountVectorizer

In [30]:
roc_auc = roc_auc_score(y_test, y_scores)
print("ROC_AUC Score:", roc_auc)

ROC_AUC Score: 0.92969088055387


In [31]:
precision = precision_score(y_test, preds)
print("Precision Score:", precision)

Precision Score: 0.8713798977853492


In [48]:
pickle.dump(precision, open('sentiment_precision_score.pickle', 'wb'))

In [32]:
recall = recall_score(y_test, preds)
print("Recall Score:", recall)

Recall Score: 0.8290113452188006


In [33]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1266
           1       0.87      0.83      0.85      1234

    accuracy                           0.86      2500
   macro avg       0.86      0.85      0.86      2500
weighted avg       0.86      0.86      0.86      2500



## Modelling / TFIDF

In [34]:
X2 = balanced_subset['description'].values
Y2 = balanced_subset['textblob'].values

In [35]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size = 0.25)

In [36]:

tfidf.fit(x_train2)
X_train2 = tfidf.transform(x_train2)
X_test2 = tfidf.transform(x_test2)

In [37]:
model2 = lr.fit(X_train2, y_train2)
model2

LogisticRegression()

In [38]:
y_scores2 = model.predict_proba(X_test2)[:, 1]

In [39]:
preds2 = model2.predict(X_test2)

## Model Scores / TFIDF

In [40]:
roc2 = roc_auc_score(y_test2, y_scores2)
precision2 = precision_score(y_test2, preds2)
recall2 = recall_score(y_test2, preds2)
print("ROC_AUC Score:", roc2)
print("Precision Score:", precision2)
print("Recall Score:", recall2)

ROC_AUC Score: 0.9397817214603017
Precision Score: 0.8862478777589134
Recall Score: 0.8358686949559647


## Score Comparisons

In [41]:
print("Precision for CountVectorizer:", precision)
print("Precision for TFIDF", precision2)

Precision for CountVectorizer: 0.8713798977853492
Precision for TFIDF 0.8862478777589134


In [42]:
from sklearn.metrics import classification_report

In [43]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1266
           1       0.87      0.83      0.85      1234

    accuracy                           0.86      2500
   macro avg       0.86      0.85      0.86      2500
weighted avg       0.86      0.86      0.86      2500

